
# 1 Introduction

Raw tabular datasets can come in many ways. How do you re-organize them for easy manipulation? How do you further prepare the data (filter, select, summarize, etc.) to get them ready for modeling? We will cover these topics in this session. 

# 2 Tidy Data

**Tidy data** is a way to organize tabular data. It provides a guide to structure and store your raw data in a consistent way for easy analysis. A table is tidy if:

1. each **variable** forms a **column**.
2. each **observation**, or **case**, forms a **row**.
3. each **type of observational unit** forms a **table**

The tidy data concept is introduced by [Wickham (2014)](http://vita.had.co.nz/papers/tidy-data.html). If you have studied Database design, you will realize that the root of the tidy data idea is Codd's [3rd normal form](https://en.wikipedia.org/wiki/Third_normal_form).

## 2.1 "Messy" data and why tidy

The tidy data definition might be too abstract. Let's start with an example of "messy" data.

|               | treatmenta  | treatmentb  |
|:-------------:|:-----------:|:-----------:|
| John Smith    | -           | 2           |
| Jane Doe      | 16          | 11          |
| Mary Johnson  | 3           | 1           |

In [ ]:
# load the tidyverse library
# this will load a set of packages
library(tidyverse)

# create a messy dataset
df_messy <- tibble(
  name = c("John Smith", "Jane Doe", "Mary Johnson"),
  treatmenta = c(NA, 16, 3),
  treatmentb = c(2, 11, 1)
)
print(df_messy)

I argue that ideally this dataset should have three variables: `name`, `treatment`, and `result`. The two column headers (`treatmenta` and `treatmentb`) in the current table should be values: treatment `a` or `b`, not variable names (i.e., it violates (1) and (2) in the tidy data definition).

The way this dataset is organized makes it hard to retrieve values and analyze them in a *consistent* way. For example, listing all treatment methods would require an operation on column names. On the other hand, listing all subject names is a column-wise operation.

In [ ]:
# list all the treatment method
print(names(df_messy))

In [ ]:
# find all the subjects
print(df_messy["name"])

### Exercise

What kind of operations (column-wise or row-wise) would you need to perform if I ask the following questions?

1. find average treatment result by person
2. find average treatment result by treatment

What if the data looks like this?

|               | John Smith  | Jane Doe  | Mary Johnson  |
|:-------------:|:-----------:|:---------:|:-------------:|
| treatmenta    | -           | 16        | 3             |
| treatmentb    | 2           | 11        | 1             |

It's just as "messy".

Imagining you want to build a set of tools/functions to analyze your data, but the raw data you collected are all messy in their own ways. Your tools/functions then need to deal with many different forms of data, which makes the workflow inefficient. That's why we would want to first store/transform raw data in/into a consistent "tidy" way.

The tidy data way.

| name          | treatment | result  |
|:-------------:|:---------:|:-------:|
| John Smith    | a         | -       |
| Jane Doe      | a         | 16      |
| Mary Johnson  | a         | 3       |
| John Smith    | b         | 2       |
| Jane Doe      | b         | 11      |
| Mary Johnson  | b         | 1       |

In [ ]:
# create a tidy dataset
df_tidy <- tibble(
  name = c("John Smith", "Jane Doe", "Mary Johnson", "John Smith", "Jane Doe", "Mary Johnson"),
  treatment = c("a", "a", "a", "b", "b", "b"),
  result = c(NA, 16, 3, 2, 11, 1)
)
print(df_tidy)

Listing all treatment methods and listing all subject names are both column-wise operations.

In [ ]:
# list all the treatment method
print(unique(df_tidy["treatment"]))

# find all the subjects
print(unique(df_tidy["name"]))

## 2.2 From messy to tidy

How do we turn a messy dataset into a tidy one? It depends on how messy the dataset is, and in what way. We'll discuss a few examples as a starter.

Let's first try to turn the messy data example seen above into a tidy one. There are many ways to do this transformation. Wickham (2014) used `melt()` function in the `reshape2` package. The `reshape2` package was replaced by [`tidyr`](https://tidyr.tidyverse.org/) package around 2015, and `gather()` function was introduced to replace `melt()`. Recently, the `tidyr` team updated `gather()` to [`pivot_longer()`](https://tidyr.tidyverse.org/reference/pivot_longer.html). Here, we will use the latest `pivot_longer()` approach.

`pivot_longer()` "'lengthens' data, increasing the number of rows and decreasing the number of columns." Precisely, it collects a set of column names and places them into a "names_to" column. It also collects the cells of those columns and places them into a "values_to" column.

In [ ]:
# tidyr is part of the tidyverse package, which we have already loaded
df_tidy_new <- pivot_longer(df_messy, -name, names_to = "treatment", values_to = "result")
print(df_tidy_new)

Let's take a look at a real world dataset for an exercise. This dataset comes with `tidyr` package. It examines the relationship between income and religion in the US.

In [ ]:
# religion vs income dataset
print(relig_income)

### Exercise
Can you tidy up the pew dataset?

In [ ]:
# your code here
# uncomment the below line to get started
# relig_income_tidy <-

The reverse operation of `pivot_longer()` is `pivot_wider()`. `pivot_wider()` "widens" data.

In [ ]:
relig_income_messy <- pivot_wider(relig_income_tidy, names_from = income, values_from = count)
print(relig_income)

`pivot_wider()` can sometimes help tidy up dateset as well. Let's take a look at the `table2` dataset provided by the `tidyr` package.

In [ ]:
print(table2)

`table2` mixes up the values of `population` and `cases` (TB cases) in the same column `count`. We can use `pivot_wider()` to tidy it up. (Discussion: Is it not tidy already?)

### Exercise
Use `pivot_wider()` to tidy up the `table2`.

In [ ]:
# your code here

## 2.3 Other messy ways

According to Wickham (2014), messy datasets have 5 common problems.

1. Column headers are values, not variable names.
2. Multiple variables are stored in one column.
3. Variables are stored in both rows and columns.
4. Multiple types of observational units are stored in the same table.
5. A single observational unit is stored in multiple tables.

We have seen (1) and (2). Let's quickly discuss another example (see slides). I'll leave you to read [(Wickham (2014)](http://vita.had.co.nz/papers/tidy-data.html) to explore the rest. This [site](https://tidyr.tidyverse.org/articles/tidy-data.html) summarizes the paper.

# 3 Data Transformation

R has many tools/packages to manipulate data. We will mainly focus on the `dplyr()` package as it's one of the best. `dplyr()` package is also part of the [`tidyverse`](https://www.tidyverse.org/packages/) eco-system. `tidyverse` consists of a set of packages that deal with data manipulation and other commom data science tasks.

Let's first load a dataset.

In [ ]:
employees <- read_csv("https://raw.githubusercontent.com/eijoac/nwdb/master/data/Employees.csv")

In [ ]:
print(employees)

In [ ]:
print(employees, n = 3, width = Inf)

## 3.1 `dplyr` basics

* filter observations: `filter()`
* select variables: `select()`
* reorder rows: `arrange()`
* create new variables: `mutate()`
* collapse column values to a single summary: `summarise()`

The above functions can be used together with `group_by()`, which changes the scope of each function from operating on the whole dataset to operating on each group.

Let's try a data manipuation task using the above functions. Find all the sales representatives who were born after 1970-01-01. Display their names and their hiring date. Name should be formatted as "LastName, FirstName". Order the output by hiring date (descending).

In [ ]:
# find all the sales representatives who were born after 1970-01-01
emp01 <- filter(employees, Title == "Sales Representative", BirthDate > 1970-01-01)
print(emp01, width = Inf)

In [ ]:
# create a new variable/column with formatted names as required
emp02 <- mutate(emp01, Name = paste(LastName, FirstName, sep = ", "))

# just print the Name variable/column to take a look at the format
print(emp02["Name"])

In the above code, [`paste()`](https://stat.ethz.ch/R-manual/R-devel/library/base/html/paste.html) is used to concatenate strings. [`glue()`](https://glue.tidyverse.org/) from the glue package might be a bit easier to use.

In [ ]:
# select the variables/columns (to be displayed)
emp03 <- select(emp02, Name, HireDate)
print(emp03)

In [ ]:
# order by HireDate
emp04 <- arrange(emp03, desc(HireDate))
print(emp04)

The above steps can be streamlined by using **pipes**.

In [ ]:
# streamline using pipes %>%
emp05 <- employees %>%
  filter(Title == "Sales Representative", BirthDate > 1970-01-01) %>%
  mutate(Name = paste(LastName, FirstName, sep = ", ")) %>%
  select(Name, HireDate) %>%
  arrange(desc(HireDate))

print(emp05)
           

### Exercise

Find all employees who are based in USA. Display their names and their birth date. Name should be formatted as "FirstName LastName". Order the output by birth date (descending).

In [ ]:
# your code here

Let's now count the number of employees by Country. We first need to group data by country using `group_by()`.

In [ ]:
emp_by_country <- group_by(employees, Country)
print(emp_by_country)

In [ ]:
summarise(emp_by_country, count = n())

In the above code, `n()` is a function for counting. Now, let's use pipe to do the same.

In [ ]:
emp_count_by_country <- employees %>%
  group_by(Country) %>%
  summarise(count = n())
print(emp_count_by_country)

Let's load another dataset for your exercise.

In [ ]:
customers <- read_csv("https://raw.githubusercontent.com/eijoac/nwdb/master/data/Customers.csv")

In [ ]:
print(customers, width = Inf)

### Exercise

Find total number of customers per Country and City. Order them by count (descending order).

In [ ]:
# your code here

Load one more dataset for another exercise.

In [ ]:
order_details <- read_csv("https://raw.githubusercontent.com/eijoac/nwdb/master/data/OrderDetails.csv")
print(order_details)

### Exercise

Find all orders with values greater than $12000. Order them by value. Ignore `Discount`. Hint: since each order (identified by `OrderID`) has many items, you need to use `group_by()`. In the `summarise()` function, use `sum()` to sum up item subtotals.

In [ ]:
# your code here

In [ ]:
# answer: method 1
order_details %>%
  group_by(OrderID) %>%
  summarise(TotalOrderValue = sum(UnitPrice * Quantity)) %>%
  filter(TotalOrderValue > 12000) %>%
  arrange(desc(TotalOrderValue))

In [ ]:
# answer: method 2
order_details %>%
  mutate(SubTotal = UnitPrice * Quantity) %>%
  group_by(OrderID) %>%
  summarise(TotalOrderValue = sum(SubTotal)) %>%
  filter(TotalOrderValue > 12000) %>%
  arrange(desc(TotalOrderValue))

## 3.2 Join tables

Most often, your dataset is organized as a collection of "tidy" tables. You need to join them for analysis. The join is based on the relationships between the tables.

This is very similar to the concept of a relational database. A database usually consists of many tables and relationships between tables. Let's see an example.

![ER](https://raw.githubusercontent.com/eijoac/nwdb/master/er_diagram/NW_ER.png)

The above graph represents 8 tables in a database and the relationships between the tables. The graph is called Entity Relationship (ER) diagram. This is a sample database (of a fake company called Northwind) from an old version of MS Access.

In the ER diagram, the tiny vertical key icon indicates a column is a primary key. A primary key is a column (or set of columns) whose values uniquely identify every row in a table. For example, `OrderID` is the primary key in the `Orders` table, and `OrderID` and `ProductID` (combined) is the primary key in the `OrderDetails` table.

The relationship icon (a line with a horizontal key at one end and an infinite symbol at the other end) indicates a foreign key constraint and a one-to-many relationship. A foreign key is a column (or set of columns) in one table whose values uniquely identify a row of another table or the same table. A foreign key mostly refers to a primary key in another table. A foreign key constraint requires that the constrained column contain only values from the primary key column of the other table. For example `CustomerID` in the `Orders` table is a foreign key that refers to the `CustomerID` primary key in the `Customers` table, and it can only contain values that exist in the `CustomerID` column of the `Customers` table.

In addition, it happens that every foreign key constraint in the Northwind DB establishes a one-to-many relationship, i.e. a row from one table can have multiple matching rows in another table. For example, one row from the `Customers` table can match multiple rows in the `Orders` table (via `CustomerID`). This makes sense as one customer can place more than one orders. (Another common relationship a foreign key constraint can establish is the one-to-one relationship.)

|logo|meaning|
|:------:|:------:|
|![key logo](https://raw.githubusercontent.com/eijoac/nwdb/master/er_diagram/key_vertical.png "key logo")|primary key|
|![foreign key constraint](https://raw.githubusercontent.com/eijoac/nwdb/master/er_diagram/relationship.png "foreign key constraint")|one-to-many foreign key constraint|

Why do we need foreign key constraints? (Discussion)

In a database, the relationships/constraints will be stored and enforced by the DB management system. Here we will just load the data as raw files to dataframes/tibbles so there is no mechanism to enforce the relationships.

Let's learn how to join datasets.

* inner join
* left / right join, and left / right join with exclusion
* full join, full outer join

Let's do some experiments using two small datasets.

In [ ]:
t1 <- tribble(
  ~pk, ~t1c1,
     1, "a",
     2, "b"
)
t2 <- tribble(
  ~fk, ~t2c1,
     1, "c",
     1, "d",
     3, "e"
)

In [ ]:
print(t1)

In [ ]:
print(t2)

In [ ]:
# playground for join exercises

# inner join
t1 %>% inner_join(t2, by = c("pk" = "fk"))

# left join
# t1 %>% left_join(t2, by = c("pk" = "fk"))

# left join with exclusion
# t1 %>% left_join(t2, by = c("pk" = "fk")) %>% filter(is.na(t2c1))

# right join
# t1 %>% right_join(t2, by = c("pk" = "fk"))

# right join with exclusion
# t1 %>% right_join(t2, by = c("pk" = "fk")) %>% filter(is.na(t1c1))

# full join
# t1 %>% full_join(t2, by = c("pk" = "fk"))

# full outer join
# t1 %>% full_join(t2, by = c("pk" = "fk")) %>% filter(is.na(t1c1) | is.na(t2c1))

### Exercise

ex1. What is `semi_join()` and `anti_join` in `dplyr()`? Is `semi_join()` the same as `inner_join`? Is `anti_join()` the same as left join with exclusion we discussed above?

In [ ]:
# test it out

In [ ]:
# inner join t1 with t2
t1 %>%
  inner_join(t2, by = c("pk" = "fk"))

# semi-join t1 with t2
t1 %>%
  semi_join(t2, by = c("pk" = "fk"))

# left join t1 with t2 and with exclusion
t1 %>%
  left_join(t2, by = c("pk" = "fk")) %>%
  filter(is.na(t2c1))

# anti-join t1 with t2
t1 %>%
  anti_join(t2, by = c("pk" = "fk"))

# make a new table
t3 <- tribble(
  ~pk, ~t1c1,
     1, "a",
     2, "b",
     2, "b"
)

# left join t3 with t2 and with exclusion
t3 %>%
  left_join(t2, by = c("pk" = "fk")) %>%
  filter(is.na(t2c1))

# anti-join t3 with t2
t3 %>%
  anti_join(t2, by = c("pk" = "fk"))

Let's load the rest of Northwind dataset and do something a bit more interesting.

In [ ]:
orders <- read_csv("https://raw.githubusercontent.com/eijoac/nwdb/master/data/Orders.csv")
shippers <- read_csv("https://raw.githubusercontent.com/eijoac/nwdb/master/data/Shippers.csv")
suppliers <- read_csv("https://raw.githubusercontent.com/eijoac/nwdb/master/data/Suppliers.csv")
products <- read_csv("https://raw.githubusercontent.com/eijoac/nwdb/master/data/Products.csv")
categories <- read_csv("https://raw.githubusercontent.com/eijoac/nwdb/master/data/Categories.csv")

ex2. Display all products and their associated suppliers.

In [ ]:
# your code here

In [ ]:
# answer
# products %>%
#   inner_join(suppliers, by = 'SupplierID')

products %>%
  left_join(suppliers, by = 'SupplierID')

ex3. Find all orders with values great than $12000 and are placed in 2016.

In [ ]:
# let's do it together
library(lubridate)
orders %>%
  filter(year(OrderDate) == 2016) %>%
  inner_join(order_details, by = "OrderID") %>%
  group_by(OrderID) %>%
  summarise(TotalOrderValue = sum(UnitPrice * Quantity)) %>%
  filter(TotalOrderValue > 12000) %>%
  arrange(desc(TotalOrderValue))

ex4. Find customers that never placed an order.

In [ ]:
# let's do it together
# method 1
customers %>%
  left_join(orders, by = "CustomerID") %>%
  filter(is.na(OrderID))

In [ ]:
# let's do it together
# method 2 - using anti_join
customers %>%
  anti_join(orders, by = "CustomerID")

ex5. Find customers who never placed an order from Margaret Peacock (EmployeeID 4).

In [ ]:
# your code here

In [ ]:
# answer: method 1
orders_4 <- orders %>%
  filter(EmployeeID == 4)

customers %>%
  anti_join(orders_4, by = "CustomerID")

In [ ]:
customers %>%
  left_join(orders_4, by = "CustomerID") %>%
  filter(is.na(OrderID))

In [ ]:
# answer: method 2
orders %>%
  filter(EmployeeID == 4) %>%
  right_join(customers, by = "CustomerID") %>%
  filter(is.na(OrderID))

ex6. Find all orders and their shippers with OrderID less than 10255. Display `OrderID`, `OrderDate` (date only), and Shipper `CompanyName`. Hint: use `as.Date()` to convert `OrderDate` (Datetime type) to Date.

In [ ]:
# let's do it together
orders %>%
  inner_join(shippers, by = c("ShipVia" = "ShipperID")) %>%
  filter(OrderID < 10255) %>%
  mutate(OrderDate = as.Date(OrderDate)) %>%
  select(OrderID, OrderDate, CompanyName)

# References

1. Tidyverse package [site](https://www.tidyverse.org/).
2. Tidy data paper intro and code [site](https://tidyr.tidyverse.org/articles/tidy-data.html).
3. Hadley Wickham's Tidy data [paper](https://www.jstatsoft.org/article/view/v059i10).
4. Garrett Grolemund's Data Tidying [tutorial](http://garrettgman.github.io/tidying/).
5. [Data Wrangle](https://r4ds.had.co.nz/wrangle-intro.html) in [R for Data Science](https://r4ds.had.co.nz/).